In [1]:
#!export PYTHONPATH=/v/xgrp0j/pnet_prostate_paper:$PYTHONPATH
!export PYTHONPATH=~/pnet_prostate_paper:$PYTHONPATH
from os.path import dirname, realpath
import sys
import kaleido
print(sys.version)
from analysis.vis_utils import get_reactome_pathway_names
#from config_path import PATHWAY_PATH
from analysis.figure_3.setup import saving_dir

current_dir = dirname(realpath("__file__"))
module_path = current_dir
import pandas as pd
import numpy as np
from os.path import join
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from os.path import dirname, realpath

from analysis.vis_utils import get_reactome_pathway_names
#from config_path import PATHWAY_PATH
from analysis.figure_3.setup import saving_dir

current_dir = dirname(realpath("__file__"))
module_path = current_dir
import pandas as pd
import numpy as np
from os.path import join
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from utils.xai_age_utils import *

3.8.8 (default, Apr 13 2021, 19:58:26) 
[GCC 7.3.0]


# Generate the Sankey diagram from the extracted folder

In [3]:

# get reactome pathway ids and names
#REACTOM_PATHWAY_PATH = "/_database/pathways/Reactome/"
reactome_pathway_df = get_reactome_pathway_names()
id_to_name_dict = dict(zip(reactome_pathway_df.id, reactome_pathway_df.name))
name_to_id_dict = dict(zip(reactome_pathway_df.name, reactome_pathway_df.id))

# nlargest= [10, 8, 8, 8, 7, 6]
nlargest = [5, 5, 5, 5, 5, 5, 5]
# nlargest= 10

node_importance = pd.read_csv(join(module_path, './extracted/node_importance_graph_adjusted.csv'), index_col=0)
#####
# node_importance.coef_combined = node_importance.coef

node_id = []
for x in node_importance.index:
    if x in name_to_id_dict.keys():
        node_id.append(name_to_id_dict[x])
    else:
        node_id.append(x)
node_importance['node_id'] = node_id

col_name = 'coef'

first_layer_nodes = node_importance[node_importance.layer == 1].copy()
other_layer_nodes = node_importance[node_importance.layer != 1].copy()
high_nodes_first_layer = get_high_nodes(first_layer_nodes, nlargest=nlargest, column='coef_combined')
# high_nodes_first_layer = get_high_nodes(first_layer_nodes, nlargest=nlargest, column='coef')
high_nodes_pathways = get_high_nodes(other_layer_nodes, nlargest=nlargest, column='coef')
# high_nodes_pathways= get_high_nodes(other_layer_nodes, nlargest=nlargest, column='coef_combined')
high_nodes = high_nodes_first_layer + high_nodes_pathways
print('high_nodes', high_nodes)
high_nodes_df = filter_nodes(node_importance, high_nodes)

high_nodes_ids = list(high_nodes_df.node_id.values)

links_df = get_links()
# links_df = get_links_with_first_layer()

'''
MDM4
'''
mdm4_nodes = get_MDM4_nodes(links_df)
mdm4_nodes_names = []
for n in mdm4_nodes:
    if n in id_to_name_dict.keys():
        mdm4_nodes_names.append(id_to_name_dict[n])
    else:
        mdm4_nodes_names.append(n)

print('mdm4_nodes', mdm4_nodes_names)

ind = links_df.source == links_df.target
links_df = links_df[~ind]

# # keep important nodes only
links_df = filter_connections(links_df, high_nodes_ids, add_unk=True)

links_df = links_df.reset_index()

# print links_df.head()
links_df['value_abs'] = links_df.value.abs()

links_df['child_sum_target'] = links_df.groupby('target').value_abs.transform(np.sum)
links_df['child_sum_source'] = links_df.groupby('source').value_abs.transform(np.sum)
links_df['value_normalized_by_target'] = 100 * links_df.value_abs / links_df.child_sum_target
links_df['value_normalized_by_source'] = 100 * links_df.value_abs / links_df.child_sum_source

#
node_importance['coef_combined_normalized_by_layer'] = 100. * node_importance[col_name] / \
                                                       node_importance.groupby('layer')[col_name].transform(np.sum)

node_importance_ = node_importance[['node_id', 'coef_combined_normalized_by_layer', col_name]].copy()
#
#
node_importance_['coef_combined_normalized_by_layer'] = np.log(
    1. + node_importance_.coef_combined_normalized_by_layer)
node_importance_normalized = node_importance_[['node_id', 'coef_combined_normalized_by_layer']]
node_importance_normalized = node_importance_normalized.set_index('node_id')
node_importance_normalized.columns = ['target_importance']
#
links_df_ = pd.merge(links_df, node_importance_normalized, left_on='target', right_index=True, how='left')
node_importance_normalized.columns = ['source_importance']
links_df_ = pd.merge(links_df_, node_importance_normalized, left_on='source', right_index=True, how='left')

#
def adjust_values(links_df_in):
    df = links_df_in.copy()
    df['A'] = df.value_normalized_by_source * df.source_importance
    df['B'] = df.value_normalized_by_target * df.target_importance
    df['value_final'] = df[["A", "B"]].min(axis=1)
    #
    df['value_old'] = df.value
    df.value = df.value_final
    #
    df['source_fan_out'] = df.groupby('source').value_final.transform(np.sum)
    df['source_fan_out_error'] = np.abs(df.source_fan_out - 100. * df.source_importance)

    df['target_fan_in'] = df.groupby('target').value_final.transform(np.sum)
    df['target_fan_in_error'] = np.abs(df.target_fan_in - 100. * df.target_importance)
    #
    #
    ind = df.source.str.contains('others')
    df['value_final_corrected'] = df.value_final
    df.loc[ind, 'value_final_corrected'] = df[ind].value_final + df[ind].target_fan_in_error
    ind = df.target.str.contains('others')

    df.loc[ind, 'value_final_corrected'] = df[ind].value_final_corrected + df[ind].source_fan_out_error

    df.value = df.value_final_corrected
    return df

df = adjust_values(links_df_)
# df.to_csv('links_df.csv')
important_node_connections_df = df.replace(id_to_name_dict)

# important_node_connections_df.to_csv('important_node_connections_df.csv')
# high_nodes_df.to_csv('high_nodes_df.csv')

high_nodes_df = high_nodes_df[[col_name, 'layer']]

# add feature nodes
high_nodes_df.loc['mutation'] = [1, 0]
high_nodes_df.loc['amplification'] = [1, 0]
high_nodes_df.loc['deletion'] = [1, 0]
high_nodes_df.loc['methylation'] = [1, 0]
# high_nodes_df.loc['other1'] = [1, 1]
# high_nodes_df.loc['hidden'] = [1, 8]

# add first layer
first_layer_df = get_first_layer_df(nlargest)
links_df = pd.concat([first_layer_df, important_node_connections_df], sort=True).reset_index()


linkes_filtred_, nodes_df = get_fromated_network(links_df, high_nodes_df, col_name=col_name, remove_others=False)

scale = 1.
width = 600. / scale
height = 0.5 * width / scale
# linkes_filtred_.to_csv('linkes_filtred.csv')
# nodes_df.to_csv('nodes_df.csv')
data_trace, layout = get_data_trace(linkes_filtred_, nodes_df, height, width)
fig = dict(data=[data_trace], layout=layout)
fig = go.Figure(fig)
filename = join(saving_dir, 'sankey_print.pdf')
fig.write_image(filename, scale=1, width=width, height=height, format='pdf')

filename = join(saving_dir, 'sankey_print.png')
fig.write_image(filename, scale=5, width=width, height=height, format='png')

from plotly.offline import plot
scale = 0.5
width = 600. / scale
height = 0.5 * width
data_trace, layout = get_data_trace(linkes_filtred_, nodes_df, height, width, fontsize=12)
fig = dict(data=[data_trace], layout=layout)
filename = 'sankey_youngblood.html'
filename = join(saving_dir, filename)
plot(fig, filename=filename)


#




/v/projects/methyldeeplearn-pallag/paurel/pnet_aging/_database/pathways/Reactome/
high_nodes ['NCOR2_cg22580512', 'RPL31_cg22809047', 'VGF_cg04084157', 'SSRP1_cg01511567', 'NHLRC1_cg22736354', 'Late Phase of HIV Life Cycle', 'Nuclear Events (kinase and transcription factor activation)', 'FGFR2 alternative splicing', 'InlA-mediated entry of Listeria monocytogenes into host cells', 'Influenza Viral RNA Transcription and Replication', 'HIV Life Cycle', 'Signaling by NTRK1 (TRKA)', 'Influenza Life Cycle', 'Signaling by FGFR2', 'Host Interactions of HIV factors', 'HIV Infection', 'Listeria monocytogenes entry into host cells', 'Signaling by NTRKs', 'Signaling by NOTCH1', 'Influenza Infection', 'Metabolism of amino acids and derivatives', 'Signaling by Receptor Tyrosine Kinases', 'Diseases of signal transduction', 'Metabolism of vitamins and cofactors', 'Signaling by NOTCH', 'Neuronal System', 'Immune System', 'Metabolism', 'Transport of small molecules', 'Metabolism of RNA']
[2, 1, 4, 5, 3,

<ipython-input-2-7641e233ce22>:174: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  high_nodes_df = high_nodes_df.append(df)


0
mdm4_nodes []
                      coef
A1BG_cg22568540   4.718241
A1CF_cg03817621   0.290398
A2M_cg12058490    0.203810
A4GNT_cg17687282  0.000000
AAAS_cg00559473   0.000000
genes_weights                            coef
A1BG_cg22568540    4.718241e+00
A1CF_cg03817621    2.903982e-01
A2M_cg12058490     2.038102e-01
A4GNT_cg17687282   0.000000e+00
AAAS_cg00559473    0.000000e+00
...                         ...
ZWILCH_cg18864581  1.008189e+00
ZWINT_cg18534872   7.060559e-01
ZWINT_cg21206959   7.402805e-01
ZZZ3_cg04127303    2.578373e-08
ZZZ3_cg24762437    5.413097e-15

[15591 rows x 1 columns]
top_genes ['NCOR2_cg22580512', 'RPL31_cg22809047', 'VGF_cg04084157', 'SSRP1_cg01511567', 'NHLRC1_cg22736354']
(5, 1)
                       coef
NCOR2_cg22580512   6.049994
RPL31_cg22809047   6.014208
VGF_cg04084157     5.937945
SSRP1_cg01511567   5.694583
NHLRC1_cg22736354  5.685286
df groupby
            source             target         value
0  gene_expression   NCOR2_cg22580512    423.11040

'~/_plots/figure3/sankey_youngblood.html'

In [5]:
plot(fig)

'temp-plot.html'

In [6]:
from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd

In [ ]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.



app = Dash(__name__)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})

fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for your data.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])

if __name__ == '__main__':
    #app.run_server(debug=True)
    app.run_server(debug=True, use_reloader=False,port=8052)  <---- Here

Dash is running on http://127.0.0.1:8052/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
